### Working 

In [1]:
import gym
import gym_sokoban
import pyglet
from pyglet import clock
import numpy as np

## Custom rendering setup if gym's rendering is not available
class Viewer:
    def __init__(self, width, height):
        self.window = pyglet.window.Window(width, height)
        self.image = None
        self.window.on_draw = self.on_draw

    def render(self, image):
        self.image = pyglet.image.ImageData(image.shape[1], image.shape[0], 'RGB', image.tobytes(), pitch=image.shape[1] * -3)
        self.window.dispatch_event('on_draw')

    def on_draw(self):
        if self.image:
            self.window.clear()
            self.image.blit(0, 0)

env = gym.make('Sokoban-small-v1')
env.reset()

# print("Room Fixed")
# print(env.room_fixed)
# print(type(env.room_fixed))
# print(env.room_fixed.shape)
# print()
# print(env.room_state)
# print()
# print(env.box_mapping)
# print()


viewer = Viewer(160, 160)  # Adjust the size according to your environment

ACTION_LOOKUP = env.unwrapped.get_action_lookup()

# Define episode and timestep parameters
num_episodes = 10
timesteps_per_episode = 100

current_episode = 0
current_timestep = 0

def update_environment(dt):
    global current_episode, current_timestep, num_episodes, timesteps_per_episode

    if current_episode < num_episodes:
        if current_timestep < timesteps_per_episode:
            # RANDOM ACTION
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            image = env.render(mode='rgb_array')
            viewer.render(image)

            print(ACTION_LOOKUP[action], reward, done, info)

            if done:
                print(f"Episode finished after {current_timestep + 1} timesteps")
                current_timestep = 0
                current_episode += 1
                env.reset()
            else:
                current_timestep += 1
        else:
            current_episode += 1
            current_timestep = 0
            env.reset()
    else:
        print("All episodes finished. Closing window.")
        viewer.window.close()  # Close the Pyglet window explicitly

# Increase the frequency to match rendering needs (e.g., 60Hz)
clock.schedule_interval(update_environment, 1/60.0)

pyglet.app.run()


/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:199: UserWarning: WARN: The result returned by `env.reset()` should be `(obs, info)` by default, , where `obs` is a observation and `info` is a dictionary containing additional information.
  logger.warn(
/home/npenchev/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API

move up -0.1 False {'action.name': 'move up', 'action.moved_player': True, 'action.moved_box': False}
push left -0.1 False {'action.name': 'push left', 'action.moved_player': False, 'action.moved_box': False}
move left -0.1 False {'action.name': 'move left', 'action.moved_player': False, 'action.moved_box': False}
move left -0.1 False {'action.name': 'move left', 'action.moved_player': False, 'action.moved_box': False}
move up -0.1 False {'action.name': 'move up', 'action.moved_player': True, 'action.moved_box': False}
move right -0.1 False {'action.name': 'move right', 'action.moved_player': True, 'action.moved_box': False}
push up -0.1 False {'action.name': 'push up', 'action.moved_player': False, 'action.moved_box': False}
push right -0.1 False {'action.name': 'push right', 'action.moved_player': True, 'action.moved_box': False}
push down -0.1 False {'action.name': 'push down', 'action.moved_player': True, 'action.moved_box': False}
no operation -0.1 False {'action.name': 'no operat

### Try to record a video

In [ ]:
import gym
import gym_sokoban
import pyglet
from pyglet import clock
import numpy as np
import cv2

## Custom rendering setup if gym's rendering is not available
class Viewer:
    def __init__(self, width, height):
        self.window = pyglet.window.Window(width, height)
        self.image = None
        self.window.on_draw = self.on_draw

    def render(self, image):
        self.image = pyglet.image.ImageData(image.shape[1], image.shape[0], 'RGB', image.tobytes(), pitch=image.shape[1] * -3)
        self.window.dispatch_event('on_draw')

    def on_draw(self):
        if self.image:
            self.window.clear()
            self.image.blit(0, 0)

env = gym.make('Sokoban-small-v1')
env.reset()

viewer = Viewer(160, 160)  # Adjust the size according to your environment

ACTION_LOOKUP = env.unwrapped.get_action_lookup()

# Define episode and timestep parameters
num_episodes = 1
timesteps_per_episode = 100

current_episode = 0
current_timestep = 0

# Video recording parameters
video_filename = 'sokoban_gameplay.mp4'
fps = 30.0

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None

def update_environment(dt):
    global current_episode, current_timestep, num_episodes, timesteps_per_episode, out

    if current_episode < num_episodes:
        if current_timestep < timesteps_per_episode:
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            image = env.render(mode='rgb_array')
            viewer.render(image)

            print(ACTION_LOOKUP[action], reward, done, info)

            if done:
                print(f"Episode finished after {current_timestep + 1} timesteps")
                current_timestep = 0
                current_episode += 1
                env.reset()
            else:
                current_timestep += 1
        else:
            current_episode += 1
            current_timestep = 0
            env.reset()
    else:
        print("All episodes finished. Closing window and saving video.")
        viewer.window.close()  # Close the Pyglet window explicitly
        out.release()  # Release the video writer

def record_frame(dt):
    global out

    # Capture the current frame from Pyglet window
    frame = pyglet.image.get_buffer_manager().get_color_buffer().get_image_data()

    # Convert frame to OpenCV format (numpy array)
    data = frame.get_data('RGB', frame.width * 3)
    arr = np.frombuffer(data, dtype=np.uint8)
    arr = arr.reshape((frame.height, frame.width, 3))

    # Convert RGB to BGR (OpenCV uses BGR)
    bgr_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)

    # Initialize video writer if not already initialized
    if out is None:
        out = cv2.VideoWriter(video_filename, fourcc, fps, (frame.width, frame.height))

    # Write frame to video
    out.write(bgr_frame)

# Schedule the frame recording function to run at a fixed interval (e.g., 30 fps)
clock.schedule_interval(record_frame, 1/fps)

# Increase the frequency to match rendering needs (e.g., 60Hz)
clock.schedule_interval(update_environment, 1/60.0)

pyglet.app.run()
